## Assignment 3

*100 points (8% of course grade)*</br>
*Assigned: Fri, Jun 20th*</br>
**Due: Fri, July 4th, 23:59**

This homework requires some SQL and Python programming, and you may need to spend some time in setting up your Postgres and Python environment. Many programming details are not covered in the lectures, so you will need to read the documentations yourself. If you wait until the last minute, you might be overwhelmed.

You must turn in the required files electronically, including a few .sql and .py files. Please follow the submission instructions at the end of this notebook.

In this assignment, you need to solve two problems. In Problem 1, you will write a few SQL queries to create and modify your database, and test your triggers. In Problem 2, you will write a few Python functions in Flask to provide web APIs for querying and modifying your database.

## Problem 1: SQL database creation (55%)

Consider the following tables storing information about the Pie-in-the-Sky Security Corp. (PITS):
* Person (<u>pid</u>, name, address); Broker (<u>pid</u>, license, phone, manager);
* Account (<u>aid</u>, brokerpid); Owns (<u>pid</u>, <u>aid</u>);
* Stock (<u>sym</u>, price);
* Trade (<u>aid</u>, <u>seq</u>, type, timestamp, sym, shares, price).

Primary keys are underlined. Note that Broker.manager can be NULL, because a boss would not have a manager.

Your job is to complete and test an implementation of the above schema design for a SQL database. To get started, copy the template files to a directory in your workspace and check that everything is in order (you may replace `~/A3/p1/` below with any other appropriate path):
```bash
cd ~/A3/p1/
ls
```

##### **IMPORTANT If you are using the course Docker container:** you will need to modify the configuration and use some slightly different commands for A3.
Please refer to the [instruction on Canvas](https://canvas.sfu.ca/courses/90697/pages/5-instructions-for-using-the-docker-container-in-a3) if you will use the course Docker container.


You should see a few `.sql` files. The file `create.sql` contains SQL statements to create the database schema. It is actually incomplete. Your first job is to edit this file to enforce a number of constraints below. You may modify the CREATE statements in the file as you see fit, but do not introduce new columns, tables, views, or triggers unless instructed otherwise. If there are comments in the file with instructions on where your edits should go, follow them. Please note:
- Use simple SQL constructs as much as possible, and only those supported by PostgreSQL.
- PostgreSQL does not allow subqueries in **CHECK**.
- PostgreSQL does not support **CREATE ASSERTION**.
- In PostgreSQL, date-time values (of type TIMESTAMP) can be represented by string literals of format, e.g., '2000-01-01 12:30:00'. These values can be compared using <, <=, =, etc., with expected semantics.
- PostgreSQL’s implementation of triggers deviates from the SQL standard. In particular, you will need to define a "UDF" (user-defined function) to execute as the trigger body. In order to complete this problem, you will need to consult the documentation at https://www.postgresql.org/docs/14/plpgsql-trigger.html and take a look at the [trigger examples](https://www.postgresql.org/docs/14/plpgsql-trigger.html#PLPGSQL-TRIGGER-EXAMPLE) in the doc.
- Particularly useful are special variables such as **NEW, TG_OP, TG_TABLE_NAME**, as well as the **RAISE EXCEPTION** statement.
- You may find these SQL functions useful: [CASE](https://www.postgresql.org/docs/14/functions-conditional.html#FUNCTIONS-CASE) and [CAST](https://www.postgresql.org/docs/14/sql-expressions.html#SQL-SYNTAX-TYPE-CASTS).
- You can enforce constraints either by rejecting database modifications that violate constraints, or by silently ignoring, changing, or augmenting the database modifications to keep constraints. For this problem, we want you to take the first approach of rejecting them (the first option in lecture). That means e.g., using RAISE EXCEPTION in triggers, and NOT using options such as DELETE ON CASCADE.



##### The following are the constraints that you need to consider ((a) is already implemented): 
- (a) Enforce key and foreign key constraints implied by the description.
- (b) Enforce that the type of a trade is either 'buy' or 'sell'.
- (c) There is no room for mistakes at PITS. Since PITS records only completed trades, enforce that the Trade table is append-only (i.e., no DELETE or UPDATE is allowed) using a trigger. Further enforce that within each account, trades must be recorded sequentially over time: i.e., compared with old trades in the same account, a new trade must have a seq that is strictly larger, and a timestamp that is no less than the old values.
- (d) Using triggers, enforce that brokers cannot own accounts, either by themselves or jointly with others.
- (e) Define a view Holds (aid, sym, amount) that returns the current account holdings, computed from the Trade table. You may assume that all accounts start with holding nothing and all transactions are recorded in Trade. If implemented correctly, your Holds view will help enforce the constraint that *one cannot sell more shares from an account than what the account actually holds*.

For (b), add constraints to the CREATE statements in `create.sql`; for (c), (d), and (e), locate all `YOUR IMPLEMENTATION GOES HERE >>>` in `create.sql` and fill in your code. Only edit the code between `YOUR IMPLEMENTATION GOES HERE >>>` and `<<< YOUR IMPLEMENTATION ENDS HERE`. **Each trigger only requires a few lines of code.**


To test create.sql, use the following commands in your terminal to (re)create a database called `pits`, and to populate it with some initial data (**Linux and MacOS**):
```bash
dropdb -U postgres pits; createdb -U postgres pits; psql pits -U postgres -af create.sql
psql pits -U postgres -af load.sql
```
You may need to replace `postgres` with your PostgreSQL username in these commands.

**For Windows:**
```bash
dropdb -d pits -U postgres; createdb -d pits -U postgres; 
psql -d pits -U postgres -f create.sql
psql -d pits -U postgres -f load.sql
```




Your next job is to write a series of SQL modification statements to test the constraints you implemented, starting with the initial data provided in `load.sql` (do not modify this file). 

You can use `psql pits` to run PostgreSQL’s interpreter interactively to experiment with your modification statements. As soon as you get a working solution for each part of this problem, say (f), record your statement in the given plain-text files (`f.sql`, `g.sql`, ..., `l.sql`).
- (f) Write an INSERT statement on Account that fails because the account’s brokerpid doesn’t refer to an existing broker.
- (g) Write a DELETE statement on Broker that fails because the broker being deleted is managing somebody else.
- (h) Write an INSERT statement that fails for violating (b).
- (i) Write a DELETE statement that fails for violating (c).
- (j) Write an INSERT statement that fails for violating (c).
- (k) Write an UPDATE statement on Broker that fails for violating (d).
- (l) Write an INSERT statement on Trade that fails for violating (e).


If your statements worked, you should see some error messages when you run the sql files, e.g., when running `psql pits -U postgres -af f.sql`
you will see `psql:f.sql:3: ERROR:  insert or update on table "account" violates foreign key constraint ...` (details skipped).

## Problem 2. Basic application programming (45%)

Many websites provide a web API for developers to use their services. Examples include [Twitter API](https://developer.twitter.com/en/docs/api-reference-index) and [Square API](https://developer.squareup.com/reference/square). In this part, your job is to implement a Web API to assist the trading in PITS. **Be sure to use SQL transactions when appropriate**. 


### Preliminary: Flask Tutorials

You need to know some basic knowledge about Flask. Flask is a (micro) web framework written in Python. It's quite easy to learn and widely used in practice (RATest also uses Flask!). 

**Step 0.** Setup your Python environment (basic installation instructions was [given in A1](https://canvas.sfu.ca/courses/90697/pages/3-dot-2-radb-system-setup-instruction-w-slash-o-docker), you only need to change it to a newer Python version). We've tested with `python 3.11.8 pip install flask==3.1.1 flask-sqlalchemy==3.1.1 psycopg2-binary==2.9.9` and `python 3.9.18, flask==3.0.3 flask-sqlalchemy==3.0.3 psycopg2-binary==2.9.3`

  * In your terminal, after you activate the run:
    ```bash
    pip install flask==3.1.1 flask-sqlalchemy==3.1.1 psycopg2-binary==2.9.9
    ```
  * you may specify the versions for each library by [library==version] or skip it for the current versions

**Step 1.** Read [A Minimal Application](https://flask.palletsprojects.com/en/3.0.x/quickstart/). Enter the `A3/p2` folder in your terminal and run the example code by `python3 flask_mini_example.py`. Make sure you can see 'Hello, World!' when opening http://127.0.0.1:5000/ in your browser. Then exit your application by `Ctrl+C` in terminal. 

**Step 2.** You will need the database in P1 (it is okay if you only run the provided `create.sql` and `load.sql`). Replace the Postgres username and password with yours in line 8 and 9 in `app.py`

**Step 3.** Enter the `A3/p2` folder and open a terminal. Run `python app.py`. Open the following URL in your browser and make sure you can see the response `["AAPL", 177.56]`.
  * http://127.0.0.1:5000/
  * If you find the error `Access to localhost was denied` in your browser, Google for `kill the process running on port` for your platform.
  
**Step 4.** Read the code in `app.py` and make sure you understand why the response of the above URL is `["AAPL", 177.56]`.

* Once you understand how `app.py` works, there is no need to learn Flask's other features for this assignment. 
* **The function `index()` in `app.py` (also the commented alternative version) provides one example of how to execute SQL queries with SQLAlchemy in Flask.** It includes the features your need except for transactions; you don't have to follow it if you know different ways to implement. You may find more [documentations for connections and transactions in SQLAlchemy](https://docs.sqlalchemy.org/en/20/core/connections.html#using-transactions)

### Tasks: finish a, b, and c below.

You only need to write about 70 lines of Python code.

You may create other functions you need and don't have to put everything in the required functions. But **DON'T add more files or refer to other Python libraries**.


#### a. Provide a method to get the owner (pid) of a given account (aid). 

Complete the `getOwner()` function in `app.py`. Follow the instructions in the comments.

**Test**

* http://127.0.0.1:5000/getOwner?aid=800 Response: [{"pid": 8008008000}]
* http://127.0.0.1:5000/getOwner?aid=150 Response: [{"pid": 1001001000}, {"pid": 5005005000}]
* http://127.0.0.1:5000/getOwner?aid=1000 Response: [{"pid": -1}]

#### b. Provide a method to get the current account holdings of a given account (aid) for a given stock (sym). 
If the stock or the account does not exist, return `{'shares': -1}`;
if the account does not hold any share of the stock, return `{'shares': 0}`.
Otherwise returns `{'shares': total_share}`.
Do NOT use the view you defined in P1.

Complete the `getHoldings()` function in in `app.py`. Follow the instructions in the comments.

**Test**

* http://127.0.0.1:5000/getHoldings?aid=150&sym=AAPL Response: {"shares": 4.00}
* http://127.0.0.1:5000/getHoldings?aid=1000&sym=AAPL Response: {'shares': -1}
* http://127.0.0.1:5000/getHoldings?aid=800&sym=AAPL Response: {'shares': 0}

#### c. Provide a method to complete a trade for given account (aid), stock (sym), operation (buy/sell), shares, and price. 
If the aid or sym does not exist, or if there is an oversell, returns `{'res' : 'fail'}`; otherwise, returns `{'res': the current seq}` and updates the database accordingly. Do NOT use the view and triggers you defined in P1.

Complete the `trade()` function in in `app.py`.

**Test**

* http://127.0.0.1:5000/trade?aid=300&sym=GOOGL&type=buy&shares=100.00&price=99.99 Response: {'res': 2}
* http://127.0.0.1:5000/trade?aid=138&sym=AAPL&type=sell&shares=101.00&price=200.20 Response: {'res': 'fail'}


You may also test these http requests through command line, run in a separate CLI/terminal window:
`curl http://127.0.0.1:5000/[the parameters]`

## Submission instruction

1. For problem 1, fill your solutions in `create.sql` and `f.sql` through `l.sql` (all in folder `p1`)

2. For problem 2, fill your solutions in `app.py` (in folder `p2`)

3. Compress folders `p1` and `p2` into `A3.zip` and submit on Canvas